In [1]:
# This script adds the unique FIPs code to each county demographic entry scraped from indexmundi.
# The unique FIPs code will allow for merging this demographic data with the COVID-19 case data and 
# health care quality data.

In [2]:
import pandas as pd

In [3]:
# Read in the data scraped from index_mundi
demographics_df = pd.read_csv('scrape_demographics.csv')
demographics_df


,Unnamed: 0,State,County,"Population estimates, July 1, 2019, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)","Population estimates base, April 1, 2010, (V2019)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent",...,"Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Black-owned firms, percent, 2007","American Indian- and Alaska Native-owned firms, percent, 2007","Asian-owned firms, percent, 2007","Native Hawaiian- and Other Pacific Islander-owned firms, percent, 2007","Hispanic-owned firms, percent, 2007","Women-owned firms, percent, 2007","Population per square mile, 2010","Land area in square miles, 2010"
0,0,ALABAMA,AUTAUGA COUNTY,"55,601",1.9%,"54,574","54,571",6.1%,23.7%,15.6%,...,285,"2,401",15.2%,0.0%,1.3%,0.0%,0.7%,31.7%,91.8,594.44
1,1,ALABAMA,BALDWIN COUNTY,"218,022",19.6%,"182,264","182,265",5.5%,21.6%,20.4%,...,"2,364","16,183",2.7%,0.4%,1.0%,0.0%,1.3%,27.3%,114.6,"1,589.78"
2,2,ALABAMA,BARBOUR COUNTY,"24,881",-9.4%,"27,457","27,457",5.2%,20.9%,19.4%,...,147,"1,445",0.0%,0.0%,0.0%,0.0%,0.0%,27.0%,31.0,884.88
3,3,ALABAMA,BIBB COUNTY,"22,400",-2.3%,"22,920","22,915",5.7%,20.5%,16.5%,...,288,967,14.9%,0.0%,0.0%,0.0%,0.0%,0.0%,36.8,622.58
4,4,ALABAMA,BLOUNT COUNTY,"57,840",0.9%,"57,321","57,322",6.0%,23.2%,18.2%,...,353,"4,030",0.0%,0.0%,0.0%,0.0%,0.0%,23.2%,88.9,644.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,WYOMING,SWEETWATER COUNTY,"43,051",-1.7%,"43,806","43,806",6.9%,26.2%,12.1%,...,307,"2,890",0.0%,0.0%,0.7%,0.0%,3.8%,27.2%,4.2,"10,426.65"
3139,3139,WYOMING,TETON COUNTY,"23,081",8.4%,"21,298","21,294",4.9%,18.4%,15.4%,...,447,"5,362",0.0%,0.0%,0.5%,0.0%,3.3%,25.3%,5.3,"3,995.38"
3140,3140,WYOMING,UINTA COUNTY,"20,299",-3.9%,"21,121","21,118",7.3%,28.8%,14.1%,...,213,"1,651",0.0%,0.0%,0.0%,0.0%,2.2%,15.9%,10.1,"2,081.26"
3141,3141,WYOMING,WASHAKIE COUNTY,"7,885",-7.5%,"8,528","8,533",5.1%,22.7%,21.7%,...,76,685,0.0%,0.0%,0.0%,0.0%,0.0%,26.9%,3.8,"2,238.55"


In [4]:
# Now we'll 'encode' the demographic data with FIPs county codes

# Read in the data called from the Census API
# https://api.census.gov/data/2010/dec/sf1?get=NAME&for=county:* => gives FIPS code for every county in the US
fips_df = pd.read_json('api_fips.json')
# Drop Puerto Rico
idx = fips_df[fips_df[1]=="72"].index
fips_df.drop(idx, inplace=True)
fips_df.columns = ['name', 'state_code', 'county_code']

In [5]:
#merge state and county code
fips_df['fips_code'] = fips_df['state_code'] + fips_df['county_code']
fips_df

,name,state_code,county_code,fips_code
0,NAME,state,county,statecounty
1,"Sebastian County, Arkansas",05,131,05131
2,"Sevier County, Arkansas",05,133,05133
3,"Sharp County, Arkansas",05,135,05135
4,"Stone County, Arkansas",05,137,05137
...,...,...,...,...
3217,"Eau Claire County, Wisconsin",55,035,55035
3218,"Florence County, Wisconsin",55,037,55037
3219,"Fond du Lac County, Wisconsin",55,039,55039
3220,"Forest County, Wisconsin",55,041,55041


In [6]:
#delete json header
fips_df = fips_df.drop(0)

#make name uppercase
fips_df['name'] = fips_df['name'].str.upper() 

fips_df

,name,state_code,county_code,fips_code
1,"SEBASTIAN COUNTY, ARKANSAS",05,131,05131
2,"SEVIER COUNTY, ARKANSAS",05,133,05133
3,"SHARP COUNTY, ARKANSAS",05,135,05135
4,"STONE COUNTY, ARKANSAS",05,137,05137
5,"UNION COUNTY, ARKANSAS",05,139,05139
...,...,...,...,...
3217,"EAU CLAIRE COUNTY, WISCONSIN",55,035,55035
3218,"FLORENCE COUNTY, WISCONSIN",55,037,55037
3219,"FOND DU LAC COUNTY, WISCONSIN",55,039,55039
3220,"FOREST COUNTY, WISCONSIN",55,041,55041


In [7]:
#Replace the '-' that is in two word states, ie 'West-Virginia' => 'West Virginia'
demographics_df['State'] = demographics_df['State'].replace(regex=['-'], value=' ')


In [8]:
# Create new combined name column
demographics_df['full_name'] = demographics_df['County'] + ", " + demographics_df['State']
demographics_df

,Unnamed: 0,State,County,"Population estimates, July 1, 2019, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)","Population estimates base, April 1, 2010, (V2019)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent",...,"Nonveteran-owned firms, 2012","Black-owned firms, percent, 2007","American Indian- and Alaska Native-owned firms, percent, 2007","Asian-owned firms, percent, 2007","Native Hawaiian- and Other Pacific Islander-owned firms, percent, 2007","Hispanic-owned firms, percent, 2007","Women-owned firms, percent, 2007","Population per square mile, 2010","Land area in square miles, 2010",full_name
0,0,ALABAMA,AUTAUGA COUNTY,"55,601",1.9%,"54,574","54,571",6.1%,23.7%,15.6%,...,"2,401",15.2%,0.0%,1.3%,0.0%,0.7%,31.7%,91.8,594.44,"AUTAUGA COUNTY, ALABAMA"
1,1,ALABAMA,BALDWIN COUNTY,"218,022",19.6%,"182,264","182,265",5.5%,21.6%,20.4%,...,"16,183",2.7%,0.4%,1.0%,0.0%,1.3%,27.3%,114.6,"1,589.78","BALDWIN COUNTY, ALABAMA"
2,2,ALABAMA,BARBOUR COUNTY,"24,881",-9.4%,"27,457","27,457",5.2%,20.9%,19.4%,...,"1,445",0.0%,0.0%,0.0%,0.0%,0.0%,27.0%,31.0,884.88,"BARBOUR COUNTY, ALABAMA"
3,3,ALABAMA,BIBB COUNTY,"22,400",-2.3%,"22,920","22,915",5.7%,20.5%,16.5%,...,967,14.9%,0.0%,0.0%,0.0%,0.0%,0.0%,36.8,622.58,"BIBB COUNTY, ALABAMA"
4,4,ALABAMA,BLOUNT COUNTY,"57,840",0.9%,"57,321","57,322",6.0%,23.2%,18.2%,...,"4,030",0.0%,0.0%,0.0%,0.0%,0.0%,23.2%,88.9,644.78,"BLOUNT COUNTY, ALABAMA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,WYOMING,SWEETWATER COUNTY,"43,051",-1.7%,"43,806","43,806",6.9%,26.2%,12.1%,...,"2,890",0.0%,0.0%,0.7%,0.0%,3.8%,27.2%,4.2,"10,426.65","SWEETWATER COUNTY, WYOMING"
3139,3139,WYOMING,TETON COUNTY,"23,081",8.4%,"21,298","21,294",4.9%,18.4%,15.4%,...,"5,362",0.0%,0.0%,0.5%,0.0%,3.3%,25.3%,5.3,"3,995.38","TETON COUNTY, WYOMING"
3140,3140,WYOMING,UINTA COUNTY,"20,299",-3.9%,"21,121","21,118",7.3%,28.8%,14.1%,...,"1,651",0.0%,0.0%,0.0%,0.0%,2.2%,15.9%,10.1,"2,081.26","UINTA COUNTY, WYOMING"
3141,3141,WYOMING,WASHAKIE COUNTY,"7,885",-7.5%,"8,528","8,533",5.1%,22.7%,21.7%,...,685,0.0%,0.0%,0.0%,0.0%,0.0%,26.9%,3.8,"2,238.55","WASHAKIE COUNTY, WYOMING"


In [9]:
# Clean up some of the scraped data

# Many places have cities instead of counties
demographics_df['full_name'] = demographics_df['full_name'].replace(regex=['CITY COUNTY,'], value='CITY,')

# Except for two places in VA
demographics_df['full_name'] = demographics_df['full_name'].replace(regex=['CHARLES CITY, VIRGINIA'], value='CHARLES CITY COUNTY, VIRGINIA')
demographics_df['full_name'] = demographics_df['full_name'].replace(regex=['JAMES CITY, VIRGINIA'], value='JAMES CITY COUNTY, VIRGINIA')


# Many places have parishes instead of counties
demographics_df['full_name'] = demographics_df['full_name'].replace(regex=['PARISH COUNTY,'], value='PARISH,')

# Alaska doesn't have counties
demographics_df['full_name'] = demographics_df['full_name'].replace(regex=[' COUNTY, ALASKA'], value=', ALASKA')

# Represent the ñ as ? because python can't handle it
demographics_df['full_name'] = demographics_df['full_name'].replace(regex=['DO±A ANA COUNTY, NEW MEXICO'], value='DO?A ANA COUNTY, NEW MEXICO')

# Drop where State and County are the same, error in scraping website where full states' information was listed with counties
identical_indices = demographics_df[ (demographics_df['State']==demographics_df['County']) & (demographics_df["State"]!='DISTRICT OF COLUMBIA')].index
demographics_df.drop(identical_indices, inplace=True)

demographics_df

,Unnamed: 0,State,County,"Population estimates, July 1, 2019, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)","Population estimates base, April 1, 2010, (V2019)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent",...,"Nonveteran-owned firms, 2012","Black-owned firms, percent, 2007","American Indian- and Alaska Native-owned firms, percent, 2007","Asian-owned firms, percent, 2007","Native Hawaiian- and Other Pacific Islander-owned firms, percent, 2007","Hispanic-owned firms, percent, 2007","Women-owned firms, percent, 2007","Population per square mile, 2010","Land area in square miles, 2010",full_name
0,0,ALABAMA,AUTAUGA COUNTY,"55,601",1.9%,"54,574","54,571",6.1%,23.7%,15.6%,...,"2,401",15.2%,0.0%,1.3%,0.0%,0.7%,31.7%,91.8,594.44,"AUTAUGA COUNTY, ALABAMA"
1,1,ALABAMA,BALDWIN COUNTY,"218,022",19.6%,"182,264","182,265",5.5%,21.6%,20.4%,...,"16,183",2.7%,0.4%,1.0%,0.0%,1.3%,27.3%,114.6,"1,589.78","BALDWIN COUNTY, ALABAMA"
2,2,ALABAMA,BARBOUR COUNTY,"24,881",-9.4%,"27,457","27,457",5.2%,20.9%,19.4%,...,"1,445",0.0%,0.0%,0.0%,0.0%,0.0%,27.0%,31.0,884.88,"BARBOUR COUNTY, ALABAMA"
3,3,ALABAMA,BIBB COUNTY,"22,400",-2.3%,"22,920","22,915",5.7%,20.5%,16.5%,...,967,14.9%,0.0%,0.0%,0.0%,0.0%,0.0%,36.8,622.58,"BIBB COUNTY, ALABAMA"
4,4,ALABAMA,BLOUNT COUNTY,"57,840",0.9%,"57,321","57,322",6.0%,23.2%,18.2%,...,"4,030",0.0%,0.0%,0.0%,0.0%,0.0%,23.2%,88.9,644.78,"BLOUNT COUNTY, ALABAMA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,WYOMING,SWEETWATER COUNTY,"43,051",-1.7%,"43,806","43,806",6.9%,26.2%,12.1%,...,"2,890",0.0%,0.0%,0.7%,0.0%,3.8%,27.2%,4.2,"10,426.65","SWEETWATER COUNTY, WYOMING"
3139,3139,WYOMING,TETON COUNTY,"23,081",8.4%,"21,298","21,294",4.9%,18.4%,15.4%,...,"5,362",0.0%,0.0%,0.5%,0.0%,3.3%,25.3%,5.3,"3,995.38","TETON COUNTY, WYOMING"
3140,3140,WYOMING,UINTA COUNTY,"20,299",-3.9%,"21,121","21,118",7.3%,28.8%,14.1%,...,"1,651",0.0%,0.0%,0.0%,0.0%,2.2%,15.9%,10.1,"2,081.26","UINTA COUNTY, WYOMING"
3141,3141,WYOMING,WASHAKIE COUNTY,"7,885",-7.5%,"8,528","8,533",5.1%,22.7%,21.7%,...,685,0.0%,0.0%,0.0%,0.0%,0.0%,26.9%,3.8,"2,238.55","WASHAKIE COUNTY, WYOMING"


In [10]:
# Need to add missing data to scraped data
# Pulled directly from Census quickfacts website, 8 counties missing
missing_df = pd.read_csv('missing_demographic_data.csv')
missing_df['full_name'] = missing_df['County'] + ", " + missing_df['State']


missing_df

,County,State,"Population estimates, July 1, 2019, (V2019)","Population estimates, July 1, 2018, (V2018)","Population estimates base, April 1, 2010, (V2019)","Population estimates base, April 1, 2010, (V2018)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2018, (V2018)","Population, Census, April 1, 2010","Persons under 5 years, percent",...,"All firms, 2012","Men-owned firms, 2012","Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010",full_name
0,HAWAII COUNTY,HAWAII,"201,513","200,983","185,076","185,076",8.90%,8.60%,"185,079",5.90%,...,"18,200","9,502","6,260","9,039","8,084","1,836","15,418",45.9,"4,028.42","HAWAII COUNTY, HAWAII"
1,UTAH COUNTY,UTAH,"636,235","622,213","516,639","516,639",23.10%,20.40%,"516,564",9.50%,...,"47,840","24,359","13,891","4,196","41,982","3,003","42,366",257.8,"2,003.45","UTAH COUNTY, UTAH"
2,OKLAHOMA COUNTY,OKLAHOMA,"797,434","792,582","718,385","718,377",11.00%,10.30%,"718,633",7.50%,...,"73,768","38,501","25,176","18,837","51,300","8,030","61,066","1,013.80",708.82,"OKLAHOMA COUNTY, OKLAHOMA"
3,IOWA COUNTY,IOWA,"16,184","16,141","16,356","16,355",-1.10%,-1.30%,"16,355",5.90%,...,"1,516",736,544,39,"1,366",104,"1,283",27.9,586.46,"IOWA COUNTY, IOWA"
4,NEW YORK COUNTY,NEW YORK,"1,628,706","1,628,701","1,586,381","1,586,360",2.70%,2.70%,"1,585,873",4.70%,...,"315,399","171,730","114,896","99,763","198,433","18,120","280,398","69,467.50",22.83,"NEW YORK COUNTY, NEW YORK"
5,ARKANSAS COUNTY,ARKANSAS,"17,486","17,769","19,007","19,007",-8.00%,-6.50%,"19,019",6.60%,...,"1,575",806,458,220,"1,281",163,"1,250",19.2,988.77,"ARKANSAS COUNTY, ARKANSAS"
6,IDAHO COUNTY,IDAHO,"16,667","16,513","16,267","16,267",2.50%,1.50%,"16,267",4.70%,...,"1,574",707,307,40,"1,467",222,"1,174",1.9,"8,477.35","IDAHO COUNTY, IDAHO"


In [11]:
# Append missing data to demographics data
demographics_df = demographics_df.append(missing_df)
demographics_df

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,"All firms, 2012","American Indian and Alaska Native alone, percent","American Indian- and Alaska Native-owned firms, percent, 2007","Asian alone, percent","Asian-owned firms, percent, 2007","Bachelor's degree or higher, percent of persons age 25 years+, 2014-2018","Black or African American alone, percent","Black-owned firms, percent, 2007","Building permits, 2018",County,...,"Two or More Races, percent",Unnamed: 0,"Veteran-owned firms, 2012","Veterans, 2014-2018","White alone, not Hispanic or Latino, percent","White alone, percent","With a disability, under age 65 years, percent, 2014-2018","Women-owned firms, 2012","Women-owned firms, percent, 2007",full_name
0,"2,949",0.5%,0.0%,1.2%,1.3%,27.7%,19.6%,15.2%,185,AUTAUGA COUNTY,...,1.8%,0.0,285,"5,071",74.3%,76.7%,14.1%,"1,093",31.7%,"AUTAUGA COUNTY, ALABAMA"
1,"19,732",0.8%,0.4%,1.2%,1.0%,31.3%,8.9%,2.7%,"3,047",BALDWIN COUNTY,...,1.8%,1.0,"2,364","19,354",83.1%,87.3%,9.1%,"6,505",27.3%,"BALDWIN COUNTY, ALABAMA"
2,"1,687",0.7%,0.0%,0.5%,0.0%,12.2%,48.4%,0.0%,6,BARBOUR COUNTY,...,1.2%,2.0,147,"1,561",45.6%,49.1%,15.8%,760,27.0%,"BARBOUR COUNTY, ALABAMA"
3,"1,310",0.4%,0.0%,0.2%,0.0%,11.5%,21.3%,14.9%,13,BIBB COUNTY,...,1.1%,3.0,288,"1,278",74.6%,76.8%,11.8%,254,0.0%,"BIBB COUNTY, ALABAMA"
4,"4,550",0.7%,0.0%,0.3%,0.0%,12.6%,1.6%,0.0%,11,BLOUNT COUNTY,...,1.4%,4.0,353,"3,765",86.9%,95.9%,9.5%,"1,400",23.2%,"BLOUNT COUNTY, ALABAMA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,"73,768",4.30%,NaN,3.60%,NaN,31.80%,15.80%,NaN,"4,070",OKLAHOMA COUNTY,...,5.40%,NaN,"8,030","51,370",55.70%,70.70%,10.00%,"25,176",NaN,"OKLAHOMA COUNTY, OKLAHOMA"
3,"1,516",0.30%,NaN,0.50%,NaN,19.10%,0.70%,NaN,13,IOWA COUNTY,...,1.10%,NaN,104,"1,102",94.60%,97.40%,7.80%,544,NaN,"IOWA COUNTY, IOWA"
4,"315,399",1.20%,NaN,12.80%,NaN,60.80%,17.90%,NaN,"3,584",NEW YORK COUNTY,...,3.40%,NaN,"18,120","32,519",47.00%,64.50%,6.10%,"114,896",NaN,"NEW YORK COUNTY, NEW YORK"
5,"1,575",0.40%,NaN,0.80%,NaN,15.50%,24.90%,NaN,18,ARKANSAS COUNTY,...,1.80%,NaN,163,"1,113",69.50%,72.10%,19.40%,458,NaN,"ARKANSAS COUNTY, ARKANSAS"


In [12]:
# join the two dataframes with a left join
# this will keep every row in the left dataframe (demographics data). When there are missing values of the 'on variable'
encoded_df = pd.merge(demographics_df, fips_df[['fips_code', 'name']], left_on='full_name', right_on='name', how='left')
encoded_df

,"All firms, 2012","American Indian and Alaska Native alone, percent","American Indian- and Alaska Native-owned firms, percent, 2007","Asian alone, percent","Asian-owned firms, percent, 2007","Bachelor's degree or higher, percent of persons age 25 years+, 2014-2018","Black or African American alone, percent","Black-owned firms, percent, 2007","Building permits, 2018",County,...,"Veteran-owned firms, 2012","Veterans, 2014-2018","White alone, not Hispanic or Latino, percent","White alone, percent","With a disability, under age 65 years, percent, 2014-2018","Women-owned firms, 2012","Women-owned firms, percent, 2007",full_name,fips_code,name
0,"2,949",0.5%,0.0%,1.2%,1.3%,27.7%,19.6%,15.2%,185,AUTAUGA COUNTY,...,285,"5,071",74.3%,76.7%,14.1%,"1,093",31.7%,"AUTAUGA COUNTY, ALABAMA",01001,"AUTAUGA COUNTY, ALABAMA"
1,"19,732",0.8%,0.4%,1.2%,1.0%,31.3%,8.9%,2.7%,"3,047",BALDWIN COUNTY,...,"2,364","19,354",83.1%,87.3%,9.1%,"6,505",27.3%,"BALDWIN COUNTY, ALABAMA",01003,"BALDWIN COUNTY, ALABAMA"
2,"1,687",0.7%,0.0%,0.5%,0.0%,12.2%,48.4%,0.0%,6,BARBOUR COUNTY,...,147,"1,561",45.6%,49.1%,15.8%,760,27.0%,"BARBOUR COUNTY, ALABAMA",01005,"BARBOUR COUNTY, ALABAMA"
3,"1,310",0.4%,0.0%,0.2%,0.0%,11.5%,21.3%,14.9%,13,BIBB COUNTY,...,288,"1,278",74.6%,76.8%,11.8%,254,0.0%,"BIBB COUNTY, ALABAMA",01007,"BIBB COUNTY, ALABAMA"
4,"4,550",0.7%,0.0%,0.3%,0.0%,12.6%,1.6%,0.0%,11,BLOUNT COUNTY,...,353,"3,765",86.9%,95.9%,9.5%,"1,400",23.2%,"BLOUNT COUNTY, ALABAMA",01009,"BLOUNT COUNTY, ALABAMA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,"73,768",4.30%,NaN,3.60%,NaN,31.80%,15.80%,NaN,"4,070",OKLAHOMA COUNTY,...,"8,030","51,370",55.70%,70.70%,10.00%,"25,176",NaN,"OKLAHOMA COUNTY, OKLAHOMA",40109,"OKLAHOMA COUNTY, OKLAHOMA"
3139,"1,516",0.30%,NaN,0.50%,NaN,19.10%,0.70%,NaN,13,IOWA COUNTY,...,104,"1,102",94.60%,97.40%,7.80%,544,NaN,"IOWA COUNTY, IOWA",19095,"IOWA COUNTY, IOWA"
3140,"315,399",1.20%,NaN,12.80%,NaN,60.80%,17.90%,NaN,"3,584",NEW YORK COUNTY,...,"18,120","32,519",47.00%,64.50%,6.10%,"114,896",NaN,"NEW YORK COUNTY, NEW YORK",36061,"NEW YORK COUNTY, NEW YORK"
3141,"1,575",0.40%,NaN,0.80%,NaN,15.50%,24.90%,NaN,18,ARKANSAS COUNTY,...,163,"1,113",69.50%,72.10%,19.40%,458,NaN,"ARKANSAS COUNTY, ARKANSAS",05001,"ARKANSAS COUNTY, ARKANSAS"


In [13]:
# Assert that no entry failed to get a fips code
assert(len(encoded_df[encoded_df['State'].isnull()])==0)
assert(len(encoded_df[encoded_df['County'].isnull()])==0)

In [14]:
encoded_df.to_csv('fips_encoded_demographics.csv')